In [1]:
import scanpy as sc 
orig_path = sc.read_h5ad("state_.h5ad")


In [19]:
import numpy as np
knockout_mask = np.isin(orig_path.var_names,orig_path.obs["target_gene"].unique())
print(f"Ratio of perturbed genes that are present in hvg set to total set : {sum(knockout_mask)/150}")

Ratio of perturbed genes that are present in hvg set to total set : 1.0


In [ ]:
adata_hvg = orig_path[:,orig_path.var["highly_variable"]]
sc.tl.rank_genes_groups(adata_hvg, groupby="condition", method="wilcoxon")


In [25]:

adata_hvg.write_h5ad("adata_Training_hvg_.h5ad")

In [1]:
from gears import GEARS,PertData
pert_data = PertData("subset_train")
pert_data.load(data_path="subset_train/arc")
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)


Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['ctrl+SV2A' 'ctrl+ANTXR1' 'ctrl+CAMSAP2' 'ctrl+WFS1' 'ctrl+SRC'
 'ctrl+KDR' 'ctrl+ATP1B1' 'ctrl+RAB3B' 'ctrl+SNCA' 'ctrl+PMEL' 'ctrl+NREP'
 'ctrl+DZIP3' 'ctrl+LAD1']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:35
Done!
Creating dataloaders....
Done!


here1


In [2]:
pert_data.adata.X[pert_data.adata.obs.condition.values=="ctrl"]

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 46772839 stored elements and shape (38176, 1792)>

In [ ]:
import numpy as np
## indices in adata.X where expression is nonzero for perturbed genesubset, grouped by gene.
non_zeros_gene_idx = {
    cond: np.flatnonzero(pert_data.adata.X[pert_data.adata.obs['condition'].values==cond].getnnz(axis=0))
    for cond in pert_data.adata.obs['condition'].unique()
}


In [8]:
print(non_zeros_gene_idx)


{'ctrl+CHMP3': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1791,)), 'ctrl+AKT2': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1792,)), 'ctrl+SHPRH': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1789,)), 'ctrl+TMSB4X': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1792,)), 'ctrl+KLF10': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1792,)), 'ctrl+TARBP2': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1792,)), 'ctrl+KDM2B': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1792,)), 'ctrl': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1792,)), 'ctrl+CLDN6': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1791,)), 'ctrl+TCF3': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1791,)), 'ctrl+NDUFB6': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1790,)), 'ctrl+TADA1': array([   0,    1,    2, ..., 1789, 1790, 1791], shape=(1792,)), 'ctrl+MED12': array([   0,    1,    2, ..., 1789, 1790, 

In [11]:
pert_data.adata.uns["non_zeros_gene_idx"] = non_zeros_gene_idx
gears_model = GEARS(pert_data, device = 'cpu', 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')
gears_model.model_initialize(hidden_size = 64)
gears_model.train(epochs = 20, lr = 1e-3)

Downloading...
100%|██████████| 60.7M/60.7M [00:18<00:00, 3.34MiB/s]  
Extracting tar file...
Done!
Start Training...


KeyError: np.str_('ctrl+MAST2')